In [2]:
import numpy as np
import pandas as pd
import requests
import json
#pip install bs4
from bs4 import BeautifulSoup

In [5]:
#user agent string
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-Agent': useragent,
           'From': 'aa7dd@virginia.ed'}

In [6]:
url = 'https://www.rottentomatoes.com/browse/movies_in_theaters/sort:a_z?page=5'
r = requests.get(url, headers=headers)
r

<Response [200]>

In [8]:
mysoup = BeautifulSoup(r.text, 'html.parser')
#mysoup

In [ ]:
# Goal 1: get a list of hyperlinks to indvidual movies' pages

In [12]:
hreflist = mysoup.find_all('a', attrs = {'data-track': 'scores'})
hreflist[0]

<a data-qa="discovery-media-list-item-caption" data-track="scores" href="/m/a_man_of_reason" slot="caption">
<score-pairs-deprecated>
<score-icon-critics sentiment="negative" size="1" slot="criticsScoreIcon"></score-icon-critics>
<rt-text context="label" size="1" slot="criticsScore"> 14%</rt-text>
<score-icon-audience sentiment="negative" size="1" slot="audienceScoreIcon"></score-icon-audience>
<rt-text context="label" size="1" slot="audienceScore"> 15%</rt-text>
</score-pairs-deprecated>
<span class="p--small" data-qa="discovery-media-list-item-title">
          A Man of Reason
        </span>
<span class="smaller" data-qa="discovery-media-list-item-start-date">
            Opened Jul 05, 2024
          </span>
</a>

In [13]:
hreflist[0]['href']

'/m/a_man_of_reason'

In [17]:
movielinks = ['https://www.rottentomatoes.com' + x['href'] for x in hreflist]
movielinks

['https://www.rottentomatoes.com/m/a_man_of_reason',
 'https://www.rottentomatoes.com/m/a_quiet_place_day_one',
 'https://www.rottentomatoes.com/m/babes',
 'https://www.rottentomatoes.com/m/bad_boys_ride_or_die',
 'https://www.rottentomatoes.com/m/banel_and_adama',
 'https://www.rottentomatoes.com/m/beverly_hills_cop_axel_f',
 'https://www.rottentomatoes.com/m/blue_lock_the_movie_episode_nagi',
 'https://www.rottentomatoes.com/m/boneyard_2024',
 'https://www.rottentomatoes.com/m/chestnut',
 'https://www.rottentomatoes.com/m/close_encounters_of_the_third_kind',
 'https://www.rottentomatoes.com/m/daddio_2023',
 'https://www.rottentomatoes.com/m/dandelion_2024',
 'https://www.rottentomatoes.com/m/dead_whisper',
 'https://www.rottentomatoes.com/m/despicable_me_4',
 'https://www.rottentomatoes.com/m/talju',
 'https://www.rottentomatoes.com/m/escape_2023',
 'https://www.rottentomatoes.com/m/evil_does_not_exist',
 'https://www.rottentomatoes.com/m/firebrand_2023',
 'https://www.rottentomatoes

In [ ]:
## Extract data from one movie page

In [32]:
url = movielinks[3]

In [33]:
r = requests.get(url, headers=headers)
mysoup = BeautifulSoup(r.text)

In [34]:
moviedata = {}

In [35]:
moviedata['title'] = mysoup.find('title').text.replace(' | Rotten Tomatoes', '') #or .string

In [36]:
moviedata

{'title': 'Bad Boys: Ride or Die'}

In [39]:
moviedata['discription'] = mysoup.find('meta', attrs = {'name': 'description'})['content']

In [40]:
moviedata

{'title': 'Bad Boys: Ride or Die',
 'discription': "This Summer, the world's favorite Bad Boys are back with their iconic mix of edge-of-your seat action and outrageous comedy but this time with a twist: Miami's finest are now on the run."}

In [43]:
moviedata['critic_score'] = mysoup.find('rt-button', slot='criticsScore').text.strip()
moviedata['audience_score'] = mysoup.find('rt-button', slot='audienceScore').text.strip()

In [49]:
moviedata['critics_consensus'] = mysoup.find('div', 'consensus').p.text

In [51]:
cat =  mysoup.find_all('div', 'category-wrap')
cat = [x.text.strip().split('\n\n\n') for x in cat]
for c in cat:
    moviedata[c[0]] = c[1].replace('\n', '')

In [52]:
moviedata

{'title': 'Bad Boys: Ride or Die',
 'discription': "This Summer, the world's favorite Bad Boys are back with their iconic mix of edge-of-your seat action and outrageous comedy but this time with a twist: Miami's finest are now on the run.",
 'critic_score': '64%',
 'audience_score': '97%',
 'critics_consensus': "Will Smith and Martin Lawrence remain good company even when Bad Boys strains to up the ante, proving there's still life left in this high-octane franchise.",
 'Director': 'Adil El Arbi, Bilall Fallah',
 'Producer': 'Jerry Bruckheimer, Will Smith, Chad Oman, Doug Belgrad',
 'Screenwriter': 'Chris Bremner',
 'Distributor': 'Columbia Pictures',
 'Production Co': 'Columbia Pictures, Westbrook Studios, Jerry Bruckheimer Films',
 'Rating': 'R (Some Sexual References|Language Throughout|Strong Violence)',
 'Genre': 'Action, Comedy',
 'Original Language': 'English',
 'Release Date (Theaters)': 'Jun 7, 2024, Wide',
 'Box Office (Gross USA)': '$177.5M',
 'Runtime': '1h 55m',
 'Sound Mix

In [59]:
# build the spider
def moviescrape(url):
    r = requests.get(url, headers=headers)
    mysoup = BeautifulSoup(r.text, 'html.parser')

    moviedata = {}
    moviedata['title'] = mysoup.find('title').text.replace(' | Rotten Tomatoes', '') #or .string
    moviedata['discription'] = mysoup.find('meta', attrs = {'name': 'description'})['content']
    moviedata['critic_score'] = mysoup.find('rt-button', slot='criticsScore').text.strip()
    moviedata['audience_score'] = mysoup.find('rt-button', slot='audienceScore').text.strip()
    try:
        moviedata['critics_consensus'] = mysoup.find('div', 'consensus').p.text
    except:
        moviedata['critics_consensus'] = ''
    cat =  mysoup.find_all('div', 'category-wrap')
    cat = [x.text.strip().split('\n\n\n') for x in cat]
    for c in cat:
        moviedata[c[0]] = c[1].replace('\n', '')

    return moviedata

In [60]:
rt_data = [moviescrape(x) for x in movielinks]

In [61]:
rt_dataframe = pd.DataFrame.from_records(rt_data)

In [62]:
rt_dataframe

,title,discription,critic_score,audience_score,critics_consensus,Director,Producer,Screenwriter,Distributor,Production Co,Genre,Original Language,Release Date (Theaters),Release Date (Streaming),Runtime,Rating,Box Office (Gross USA),Sound Mix,Aspect Ratio,Rerelease Date (Theaters)
0,A Man of Reason,After 10 years in the prison on behalf of his ...,14%,15%,,Jung Woo-sung,Song Dae-chan,"Jung Hae-sin, Jung Woo-sung",Epic Pictures,Studio Take,"Action, Drama, Mystery & Thriller",Korean,"Jul 5, 2024, Limited","Jul 9, 2024",1h 37m,NaN,NaN,NaN,NaN,NaN
1,A Quiet Place: Day One,Experience the day the world went quiet.,87%,73%,Grounded in raw humanity by Lupita Nyong'o and...,Michael Sarnoski,"Michael Bay, Andrew Form, Brad Fuller, John Kr...",Michael Sarnoski,Paramount Pictures,"Platinum Dunes, Sunday Night","Horror, Sci-Fi, Mystery & Thriller",English,"Jun 28, 2024, Wide",NaN,1h 40m,PG-13 (Violent Content/Bloody Images|Terror),$94.0M,"Dolby Atmos, Dolby Digital",Digital 2.39:1,NaN
2,Babes,BABES follows inseparable childhood friends Ed...,91%,79%,Offering an uproariously unvarnished look at m...,Pamela Adlon,"Ilana Glazer, Josh Rabinowitz, Susie Fox, Ashl...","Ilana Glazer, Josh Rabinowitz",Neon,"FilmNation Entertainment, Starrpix, Range Medi...",Comedy,English,"May 24, 2024, Wide","Jul 2, 2024",1h 44m,R (Language Throughout|Some Drug Use|Sexual Ma...,$3.7M,NaN,NaN,NaN
3,Bad Boys: Ride or Die,"This Summer, the world's favorite Bad Boys are...",64%,97%,Will Smith and Martin Lawrence remain good com...,"Adil El Arbi, Bilall Fallah","Jerry Bruckheimer, Will Smith, Chad Oman, Doug...",Chris Bremner,Columbia Pictures,"Columbia Pictures, Westbrook Studios, Jerry Br...","Action, Comedy",English,"Jun 7, 2024, Wide",NaN,1h 55m,R (Some Sexual References|Language Throughout|...,$177.5M,"Dolby Atmos, Dolby Digital",Digital 2.39:1,NaN
4,Banel & Adama,Banel and Adama are fiercely in love. The youn...,95%,,,Ramata-Toulaye Sy,"Éric Névé, Maud Leclair, Andrey Samoute Diarra...",Ramata-Toulaye Sy,Kino Lorber,"La Chauve Souris, arte France Cinéma, Astou Films",Drama,French (France),"Jun 7, 2024, Limited","Aug 6, 2024",1h 27m,NaN,$38.5K,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,The Strangers: Chapter 1,After their car breaks down in an eerie small ...,21%,44%,Lacking the chilling suspense of the original ...,Renny Harlin,"Courtney Solomon, Mark Canton, Christopher Mil...","Alan R. Cohen, Alan Freedland, Bryan Bertino",Lionsgate,Fifth Element Productions,"Horror, Mystery & Thriller",English,"May 17, 2024, Wide","Jun 7, 2024",1h 31m,R (Language|Horror Violence|Brief Drug Use),$35.1M,NaN,Digital 2.39:1,NaN
72,The Watchers (2024),"The Watchers follows Mina, a 28-year old artis...",32%,52%,Heavy on eerie mood but undone by an obtuse sc...,Ishana Shyamalan,"M. Night Shyamalan, Ashwin Rajan, Nimitt Mankad",Ishana Shyamalan,Warner Bros. Pictures,"New Line Cinema, Inimitable Pictures, Blinding...","Horror, Mystery & Thriller",English,"Jun 7, 2024, Wide","Jun 28, 2024",1h 42m,PG-13 (Violence|Terror|Some Thematic Elements),$19.1M,Dolby Digital,Flat (1.85:1),NaN
73,Thelma (2024),Inspired by a real-life experience of director...,99%,83%,A stellar showcase for the talented June Squib...,Josh Margolin,"Zoë Worth, Chris Kaye, Nicholas Weinstock, Ben...",Josh Margolin,Magnolia Pictures,"Bandwagon, Zurich Avenue, Invention Studios","Comedy, Action",English,"Jun 21, 2024, Wide",NaN,1h 37m,PG-13 (Strong Language),$6.9M,NaN,NaN,NaN
74,Touch (2024),A romantic and thrilling story that spans seve...,100%,,,Baltasar Kormákur,"Baltasar Kormákur, Agnes Johansen, Mike Goodridge","Baltasar Kormákur, Ólaf Ólafsson",Focus Features,"RVK Studios, Good Chaos","Romance, Drama",English,"Jul 12, 2024, Limited",NaN,2h 1m,R (Some Sexuality),NaN,NaN,NaN,NaN
